In [8]:
from pyDecision.algorithm import ahp_method
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("results/filtered_responses.csv")

In [6]:
def AHP(matrix,derivation="geometric"):
    #weights
    weights,rc = ahp_method(matrix,wd=derivation)

    for i in range (0, weights.shape[0]):
        print ('w_w(feature '+str(i+1)+'): ',round(weights[i],3))

    #consistency ratio
    print('RC: ' + str(round(rc, 2)))
    if (rc > 0.1):
        print('The solution is inconsistent, pairwise comparison must be reviewed')
    else:
        print('The solution is consistent')

    return weights

In [ ]:
# decision matrix (worker,green,safer)

In [ ]:
print('Worker')
weights_w = AHP(dec_mtx_worker)
print('\nGreen')
weights_g = AHP(dec_mtx_gree)
print('\nSafe')
weights_s = AHP(dec_mtx_safer)
print('\nTourist')
weights_t = AHP(decision_matrix_tourist)

In [ ]:
# decision matrix (ldt,lpn,lca,cna) greedy

In [ ]:
print('Worker greedy')
weights_ldt = AHP(dec_mtx_ldt)
print('\nGreen greedy')
weights_lpn = AHP(dec_mtx_lpn)
print('\nSafe greedy')
weights_lca = AHP(dec_mtx_lca)
print('\nTourist greedy')
weights_cna = AHP(dec_mtx_cna)

In [ ]:
def evaluate_routes(routes, weights_w, columns):
    routes_parameters = routes[columns[1:]]
    results_array = []
    result = 0
    for route in routes_parameters.iterrows(): # iterrows !!!
        result = np.multiply(np.array(route[1]), np.array(weights_w))
        results_array.append(np.sum(result))
    return results_array

def evaluate_all(weight_list,dataframe_list,columns):
    result = []
    for i in weight_list:
        result.append([])
    
    for w in range(len(weight_list)):
        for df in dataframe_list:
            result[w].append(evaluate_routes(df,weight_list[w],columns))
    return result

def Id_SetId(result_list,dataframe_list):
    id_list = []
    for i in result_list:
        id_list.append([])
    
    for result in range(len(result_list)):
        #for weight in result_list[result]:
        for weight in range(len(result_list[result])):
            
            max_value = max(result_list[result][weight])
            max_index = result_list[result][weight].index(max_value)
            df = dataframe_list[weight].loc[max_index]
            id_list[result].append([max_value,df['ID'],df['setID']])
    return id_list

In [ ]:
dfs = []
for i in range(600):
    x = df.loc[df['setID'] == i]
    if x.empty or x.isnull().values.any():
        continue
    x = x.reset_index()
    dfs.append(x)
print("Numbers of origin - destiny 2 pairs:",len(dfs))

In [ ]:
columns = ['ID', 'crimeNorm', 'accidentNorm', 'natureNorm', 'attractionNorm', 'durationNorm',
           'trafficNorm', 'lengthNorm', 'polutionNorm'] # verify columns with dataframe

weight_list = [weights_w,weights_g,weights_s,weights_t]
r = evaluate_all(weight_list,dfs,columns)
id_list = Id_SetId(r,dfs)

In [ ]:
def sum_all(id_list,dataframe_list,columns):
    
    means = []
    
    for i in range(len(id_list)):
        
        mean_df = pd.DataFrame(columns=columns)
        
        for j in range(len(dataframe_list)):
            #x = dfs[0].loc[dfs[0]['ID'] == id_list[0][0][1]]
            mean_df = pd.concat([mean_df,dataframe_list[j].loc[dataframe_list[j]['ID'] == id_list[i][j][1]]],join='inner')
        means.append(mean_df.mean())
        
    return means

def sum_greedy(dataframe_list,columns,norms):
    
    greedy_means = []
    
    for i in norms:
        
        mean_df = pd.DataFrame(columns=columns)
        
        for j in range(len(dataframe_list)):
            
            mean_df = pd.concat([mean_df, dataframe_list[j].loc[dataframe_list[j][i] == 1]],join='inner')
            
        greedy_means.append(mean_df.mean())
    
    return greedy_means

In [ ]:
columns = ["crimeNorm","accidentNorm","natureNorm","attractionNorm","durationNorm","trafficNorm","lengthNorm","crimes",
           "accidents","attractions","nature","traffic","duration","length","polution","polutionNorm"]

mean = sum_all(id_list,dfs,columns)


norms = ["crimeNorm","accidentNorm","natureNorm","attractionNorm","durationNorm","trafficNorm","lengthNorm","polutionNorm"]

columns = ["crimeNorm","accidentNorm","natureNorm","attractionNorm","durationNorm","trafficNorm","lengthNorm","crimes",
           "accidents","attractions","nature","traffic","duration","length","polution","polutionNorm"]

greedy_means = sum_greedy(dfs,columns,norms)

In [ ]:
def PDFKS(dataframe_list,best_list,sum_name):
    
    best_list_values = []
    
    for i in range(len(sum_name)):
        best_list_values.append(best_list[i][sum_name[i]])
    
    result = []
    
    for i in range(len(dataframe_list)): # iterate over dataframe
        
        line = []
        
        for j in range(len(sum_name)): # iterate over raw data only len = 8

            df_value = dataframe_list[i][sum_name[j]]

            best_value = best_list_values[j]
            
            line.append((df_value-best_value)/best_value * 100)
            
        result.append(line)
    return result

In [ ]:
sum_name = ['crimes','accidents','nature','attractions','duration','traffic','length','polution']

result = PDFKS(mean+greedy_means,greedy_means,sum_name)

# PDFKS Heat Map Plot & Histogram

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=False, bottom=True,
                   labeltop=False, labelbottom=True)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
profiles = ['worker','green','safe','tourist','onlyCrimes','onlyAccidents','onlyNature','onlyAttractions',
            'onlyDuration','onlyTraffic','onlyLength','onlyPolution']

features = ['crimes','accidents','nature','attractions','duration','traffic','length','polution']

result_np = np.array(result)

fig, ax = plt.subplots(figsize=(16,9),dpi=80)

im, cbar = heatmap(result_np, profiles, features, ax=ax,
                   cmap="YlGn", cbarlabel="PDFKS Value",vmin=-100, vmax=100)
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
result_sum = []
for i in result:
    res = 0
    for j in i:
        res += abs(j)
    result_sum.append(res)
    
x = list(zip(profiles,result_sum))

sorted_result = sorted(x,key=lambda x: int(x[1]))

profiles_names = []
values = []
for (i,j) in sorted_result:
    profiles_names.append(i)
    values.append(j)

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(profiles_names, values, align='center')
ax.set_yticks(profiles_names, labels=profiles_names)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Absolute sum [%]')
ax.set_ylabel('Classification method')

plt.show()

In [ ]:
#features = ['crimes','accidents','nature','attractions','duration','traffic','length','polution']
#profile = ['Worker', 'Green', 'Safe', 'Tourist']
#best_values = [507757.105634,126.691275,809605.474893,4.756345,1967.288732,0.826151,15745.302817,28.212792]

#criminality
fig, ax = plt.subplots()

profile = ['Worker', 'Green', 'Safe', 'Tourist']
value = [585863.605634, 592358.915493, 526048.500000, 651023.119718]

ax.bar(profile, value)

ax.axhline(y = 507757.105634, color = 'r', linestyle = '--', label = "Best Value")
ax.set_ylabel('Crime occurance')
ax.set_title('Average Crime occurance for each profile')

plt.show()

In [ ]:
## multiple other plots 